In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
import torch
import matplotlib.pyplot as plt

**This Notebook is a review of important notions**

# Callbacks

## Callbacks as event GUI

In [2]:
import ipywidgets as widgets

In [3]:
def f(o): print('hi')

In [4]:
w = widgets.Button(description='Click')

In [5]:
w

Button(description='Click', style=ButtonStyle())

In [6]:
w.on_click(f)

[plotly](https://plot.ly/python/widget-app/)

## Creating a callback

In [7]:
from time import sleep

In [9]:
def slow_calculation():
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
    return res

In [10]:
slow_calculation()

30

Now with a callback calling:

In [11]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
        if cb: cb(i)
    return res

A simple callback func:

In [12]:
def show_prog(epoch):
    print(f"Cool! epoch {epoch}")

In [13]:
slow_calculation(show_prog)

Cool! epoch 0
Cool! epoch 1
Cool! epoch 2
Cool! epoch 3
Cool! epoch 4


30

## Lambdas and partial functions

Lambda is a simple unassigned function:

In [15]:
slow_calculation(lambda x: print(f'cool, epoch {x}'))

cool, epoch 0
cool, epoch 1
cool, epoch 2
cool, epoch 3
cool, epoch 4


30

In [21]:
def show_progress(exclamation, epoch):
    print(f"{exclamation}! We've finished epoch {epoch}!")

In [16]:
def make_show_progress(exclamation):
    _inner = lambda epoch: print(f"{exclamation}! We've finished epoch {epoch}!")
    return _inner

is the same as:

In [ ]:
def make_show_progress(exclamation):
    # Leading "_" is generally understood to be "private"
    def _inner(epoch): print(f"{exclamation}! We've finished epoch {epoch}!")
    return _inner

This is called a "closure" meaning that the function ecapsulate some of the paramaters of another function.

In [17]:
f2 = make_show_progress("Terrific")

In [18]:
slow_calculation(f2)

Terrific! We've finished epoch 0!
Terrific! We've finished epoch 1!
Terrific! We've finished epoch 2!
Terrific! We've finished epoch 3!
Terrific! We've finished epoch 4!


30

We can do it with a "partial" function:

In [19]:
from functools import partial

In [22]:
slow_calculation(partial(show_progress, "OK I guess"))

OK I guess! We've finished epoch 0!
OK I guess! We've finished epoch 1!
OK I guess! We've finished epoch 2!
OK I guess! We've finished epoch 3!
OK I guess! We've finished epoch 4!


30

## Callable classes:

But if we have a lot of parameters that we would like te register by default we should make a class:

In [24]:
class ProgressShowingCallback():
    def __init__(self, exclamation="Awesome"): 
        self.exclamation = exclamation
        
    def __call__(self, epoch): 
        print(f"{self.exclamation}! We've finished epoch {epoch}!")

In [25]:
cb = ProgressShowingCallback("Just super")

In [26]:
slow_calculation(cb)

Just super! We've finished epoch 0!
Just super! We've finished epoch 1!
Just super! We've finished epoch 2!
Just super! We've finished epoch 3!
Just super! We've finished epoch 4!


30

__ call __   allow to call the object just like a function

## args and kwargs

These are the arguments and KeyWord Arguments of a function


Kwargs allow to wrap and pass a bunch of arguments to another class object.

It's convenient for prototyping but don't uveruse it!

In [32]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        if cb: cb.before_calc(i)
        res += i*i
        sleep(1)
        if cb: cb.after_calc(i, val=res)
    return res

In [35]:
class PrintStatusCallback():
    def __init__(self): pass
    def before_calc(self, epoch, **kwargs): print(f"About to start: {epoch}")
    def after_calc (self, epoch, val, **kwargs): print(f"After {epoch}: {val}")

In [36]:
slow_calculation(PrintStatusCallback())

About to start: 0
After 0: 0
About to start: 1
After 1: 1
About to start: 2
After 2: 5
About to start: 3
After 3: 14
About to start: 4
After 4: 30


30

Still, having kwargs makes it more resilient because some time in the future we might have more arguments to pass for other callbacks and it wont break the code.

## Modify behavior

In [37]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        if cb and hasattr(cb, 'before_calc'): ## allow to check if function exist without breaking 
            cb.before_calc(i)
        res += i*i
        sleep(1)
        if cb and hasattr(cb, 'after_calc'): 
            if cb.after_calc(i,res):
                print('stopping early')
                break
    return res

In [40]:
class PrintAfterCB():
    def after_calc (self, epoch, val):
        print(f'After {epoch}: {val}')
        if val>10:
            return True

In [41]:
slow_calculation(PrintAfterCB())

After 0: 0
After 1: 1
After 2: 5
After 3: 14
stopping early


14

We can refactor slow_calculation in a class to make the call to callbacks more automated:

In [1]:
class SlowCalculator():
    def __init__(self, cb=None):
        self.cb = cb
        self.res = 0
    def __call__(self, cb_name, *args):
        # This method checks if the callback exists and then executes it
        # Notice we pass the SlowCalculator object to the callback function
        # so it can access it's parameters.
        if not self.cb: return
        cb = getattr(self.cb, cb_name, None) # this line recover the method in the cb object
        if cb:
            return cb(self, *args)
        
    def calc(self):
        for i in range(5):
            self('before_calc', i)
            self.res += i*i
            sleep(1)
            if self('after_calc', i):
                print('Stopping early')
                break
            

In [59]:
class ModifCB():
    def before_calc(self,calc, epoch):
        print('Start calc')
        
    def after_calc(self, calc, epoch):
        print(f'After {epoch}: {calc.res}')
        if calc.res>10: return True
        if calc.res<3 : calc.res = calc.res*2

In [60]:
calculator = SlowCalculator(ModifCB())

In [62]:
calculator.calc()
calculator.res

Start calc
After 0: 15
Stopping early


15

# Dunder functions:

In [63]:
class SloppyAdder():
    def __init__(self,o): self.o=o
    def __add__(self,b): return SloppyAdder(self.o + b.o + 0.01)
    def __repr__(self): return str(self.o)

In [64]:
a = SloppyAdder(1)
b = SloppyAdder(2)
a+b

3.01

Special methods you should probably know about (see data model link above) are:

- `__getitem__`
- `__getattr__`
- `__setattr__`
- `__del__`
- `__init__`
- `__new__`
- `__enter__`
- `__exit__`
- `__len__`
- `__repr__`
- `__str__`